In [ ]:
!pip install tmdbsimple

In [ ]:
import tmdbsimple as tmdb
import pandas as pd
from sentence_transformers import SentenceTransformer
from sentence_transformers.util import cos_sim
import gradio as gr
from datasets import load_dataset
import ast
from transformers import pipeline

In [ ]:
data = load_dataset("AiresPucrs/tmdb-5000-movies",split="train")
data = pd.DataFrame(data,columns=data[0].keys())
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4803 entries, 0 to 4802
Data columns (total 22 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    4803 non-null   int64  
 1   budget                4803 non-null   int64  
 2   genres                4803 non-null   object 
 3   homepage              1712 non-null   object 
 4   keywords              4803 non-null   object 
 5   original_language     4803 non-null   object 
 6   original_title        4803 non-null   object 
 7   overview              4800 non-null   object 
 8   popularity            4803 non-null   float64
 9   production_companies  4803 non-null   object 
 10  production_countries  4803 non-null   object 
 11  release_date          4802 non-null   object 
 12  revenue               4803 non-null   int64  
 13  runtime               4801 non-null   float64
 14  spoken_languages      4803 non-null   object 
 15  status               

In [ ]:
def combine(row):
  genre = ', '.join([g['name'] for g in ast.literal_eval(row['genres'])])
  keywords = ', '.join([g['name'] for g in ast.literal_eval(row['keywords'])])
  parts=[
      f'Title: {row["title"]}',
      f'Overview: {row["overview"]}',
      f'Genres: {genre}',
      f'Tagline: {row["tagline"]}',
      f'Keywords: {keywords}'
  ]
  return ' '.join(parts)
data['comb_col'] = data.apply(combine,axis=1)
data['comb_col']

"Title: Four Rooms Overview: It's Ted the Bellhop's first night on the job...and the hotel's very unusual guests are about to place him in some outrageous predicaments. It seems that this evening's room service is serving up one unbelievable happening after another. Genres: Crime, Comedy Tagline: Twelve outrageous guests. Four scandalous requests. And one lone bellhop, in his first day on the job, who's in for the wildest New year's Eve of his life. Keywords: hotel, new year's eve, witch, bet, hotel room, sperm, los angeles, hoodlum, woman director, episode film"

In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
data['encoded'] = data['comb_col'].apply(lambda x : model.encode(x))
data['encoded']

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

,encoded
0,"[0.030198835, -0.042928085, 0.0019527671, -0.0..."
1,"[-0.053609263, -0.09365986, 0.07629205, -0.092..."
2,"[-0.0577582, 0.028028816, -0.009681891, -0.032..."
3,"[-0.04178783, -0.036373362, 0.019230967, 0.018..."
4,"[0.028552236, -0.03981372, 0.009375247, 0.0683..."
...,...
4798,"[0.046840075, -0.0021982528, -0.05943704, 0.01..."
4799,"[-0.008413983, 0.051562805, -0.06678989, 0.030..."
4800,"[-0.03989293, 0.008886725, -0.041109804, 0.100..."
4801,"[-0.030443251, -0.03563389, 0.028420826, 0.014..."


In [ ]:
model = SentenceTransformer('all-MiniLM-L6-v2')
x=model.encode("hello there")
x.shape

(384,)

In [ ]:
google_flan = pipeline("text2text-generation", model="google/flan-t5-base")

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

Device set to use cpu


In [ ]:
def recommend(inp,top_k=5):
  e_inp = model.encode(inp)
  data['sim'] = data['encoded'].apply(lambda x : cos_sim(e_inp,x)[0][0].item())
  res = data.sort_values('sim',ascending=False).head(top_k+1)['title'][1:]
  results = [r for r in res]
  prompt = (
    f"User request: {inp}\n\n"
    f"The following movies were recommended based on similarity:\n{results}\n\n"
    "Please suggest these movies to the user with engaging explanations and a brief overview for each one. "
    "Make the tone friendly and informative, as if you're a helpful movie guide."
  )
  output = google_flan(prompt, max_length=200, do_sample=True)[0]['generated_text']
  return output
recommend("I want to watch a thriller with a strong female lead, something like ‘Gone Girl’ but more intense")

Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


"I want to watch a thriller with a strong female lead, something like 'Gone Girl' but more intense. The following movies were recommended based on similarity: ['Sympathy for Lady Vengeance', 'Run, Hide, Die', 'The Boy Next Door', 'Run, Hide, Die', 'The Last Exorcism Part II', 'Abandon']"

In [ ]:
def chatbot_response(message, history=None):
    reply = recommend(message, 5)
    return reply
gr.ChatInterface(
    fn=chatbot_response,
    title="🎬 Movie Recommender Bot",
    description="Ask for movie suggestions! Try: 'I want a sci-fi movie like Inception.'"
).launch(share=True,debug=True)


/usr/local/lib/python3.12/dist-packages/gradio/chat_interface.py:347: UserWarning: The 'tuples' format for chatbot messages is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style 'role' and 'content' keys.
  self.chatbot = Chatbot(


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://1378acc32560a53aed.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Both `max_new_tokens` (=256) and `max_length`(=200) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7866 <> https://1378acc32560a53aed.gradio.live
